In [27]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

df = pd.read_csv('../database/stock_code/stock_code_kospi.csv')
#df = pd.read_csv('../database/stock_code/stock_code_kosdak.csv')

def stock_price_crawl(stock_code):
    output = pd.DataFrame(columns=['날짜','종가','거래량'])
    i = 0
    while i <= 100:
        i += 1
        url = 'https://finance.naver.com/item/sise_day.nhn?code=' + str(stock_code).zfill(6) + f'&page={i}'
        temp = pd.read_html(url)[0].dropna(axis=0)[['날짜','종가','거래량']]
        if temp['날짜'].iloc[0] in output['날짜']:
            break
        else:
            output = pd.concat([output, temp], axis=0, ignore_index=True)
    output = output.rename(columns=dict(날짜='Date', 종가='Price', 거래량='Volume'))
    output['Date'] = output['Date'].apply(lambda x: x.replace('.','-'))
    output['StockCode'] = stock_code
    return output

def job_collect(df):
    output = pd.DataFrame()
    for i in tqdm(range(len(df))):
        temp = stock_price_crawl(df['StockCode'][i])
        output = pd.concat([output, temp], axis=0, ignore_index=True)
    output = output.drop_duplicates()
    output.to_csv(f'../database/stock_price/stock_price.csv', index=False)
    output.to_csv(f'../database/stock_price/stock_price_{today}.csv', index=False)  

In [28]:
job_collect(df)

100%|██████████████████████████████████████████████████████████████████████████████| 796/796 [1:21:15<00:00,  6.13s/it]
